In [ ]:
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import LineString
import numpy as np
# import contextily as cx

In [ ]:
DATA_FOLDER = Path("data/prorail")

In [ ]:
CRS = 'EPSG:28992'

# 1. Train GPS points (MTPS)

In [ ]:
df_mtps = pd.read_parquet(DATA_FOLDER / 'mtps_2023_10_02_12.parquet')

In [ ]:
def to_point_gpd(df):
    return gpd.GeoDataFrame(
        df[["longitude", "latitude"]],
        geometry=gpd.points_from_xy(df.longitude, df.latitude),
        crs=CRS,
    )

In [ ]:
df_points = to_point_gpd(df_mtps)
df_points.head()

# 2. Rail branches

In [ ]:
df_spoortak = gpd.read_parquet(DATA_FOLDER / "spoortakken.parquet").to_crs(CRS).drop_duplicates()

In [ ]:
def plot_spoortakken(df):
    fig, ax = plt.subplots(figsize=(10, 10))
    df.plot(ax=ax, color="black", linewidth=0.5, )
    cx.add_basemap(ax, crs=df_spoortak.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)
    return fig, ax

In [ ]:
# _ = plot_spoortakken(df_spoortak)

# 3. GeoPandas join performance

In [ ]:
# Further split up the tracks in 100m sections.
def segmentize(line, d=100):
    return LineString(
        [line.interpolate(l) for l in np.arange(0, line.length, d)]
        + [line.boundary.geoms[1]]
    )

def segments(curve):
    return list(map(LineString, zip(curve.coords[:-1], curve.coords[1:])))

df_spoortak['segments'] = df_spoortak.geometry.apply(lambda x: segments(segmentize(x)))

In [ ]:
# Create DataFrame for track segments. 
df_segments = df_spoortak.explode('segments')

# Add incrementing index.
df_segments['segment_id'] = df_segments.groupby('geometry').cumcount().astype(str).values

# Set segment as row geometry and select relevant rows.
df_segments.drop(columns=['geometry'], inplace=True)
df_segments.rename(columns={'segments': 'geometry', 'NAAM': 'branch_name'}, inplace=True)
df_segments.set_geometry('geometry', drop=True, inplace=True, crs=CRS)
df_segments = df_segments[['branch_name', 'geometry', 'segment_id']]

df_segments.head()

In [ ]:
# Perform single join.
df_joined = df_points.sjoin_nearest(df_segments, how="left", max_distance=4, distance_col="distance")

In [ ]:
import time

results = []

for i in [1, 6, 12, 24, 48, 72]:
    df_points = to_point_gpd(pd.concat([df_mtps] * i, ignore_index=True))

    st = time.time()
    x = df_points.sjoin_nearest(df_segments, how="left", max_distance=4, distance_col="distance")    
    et = time.time()
    del x

    results.append((len(df_points), et - st))


In [ ]:
results

# 4. (old) NCBGs

In [ ]:
# # Load NCBG polygons.
# df_ncbg = get_omgevingsvergunning_areas(DATA_FOLDER / 'regions')
# df_ncbg.head()

In [ ]:
# # Plot four NCBGs.
# REGIONS = ["Utrecht Centraal Station", "Nijmegen", "Amsterdam CS", "Eindhoven"]
# TITLES = ["Utrecht Centraal", "Nijmegen", "Amsterdam Centraal", "Eindhoven Centraal"]
# fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# for r, title, ax in zip(REGIONS, TITLES, axs.flat):
#     # Plot region polygon.
#     region = df_ncbg.where(df_ncbg.EMPLACEMEN == r)
#     region.plot(ax=ax, color="red", edgecolor="black", alpha=0.5, linewidth=1)

#     # Set title and remove labels.
#     ax.set_title(title)
#     ax.set_xticks([])
#     ax.set_yticks([])

#     # Determine bounds for square images.
#     x1, y1, x2, y2 = region.total_bounds
#     dx = x2 - x1
#     dy = y2 - y1
#     r = dx / dy

#     if r > 1:
#         y1 = y1 - (dy * (r - 1) / 2)
#         y2 = y2 + (dy * (r - 1) / 2)
#     else:
#         x1 = x1 - (dx * (1 / r - 1) / 2)
#         x2 = x2 + (dx * (1 / r - 1) / 2)

#     # Zoom out slightly.
#     dx = x2 - x1
#     dy = y2 - y1
#     zoom = 0.1

#     x1 = x1 - zoom * dx / 2
#     x2 = x2 + zoom * dx / 2
#     y1 = y1 - zoom * dy / 2
#     y2 = y2 + zoom * dy / 2

#     ax.set_xlim(x1, x2)
#     ax.set_ylim(y1, y2)

#     cx.add_basemap(
#         ax, crs=region.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik
#     )


# fig.tight_layout()